## Heuristik

Da die Rechenleistung nicht ausreicht um vor jedem Zug den gesamten Spielbaum abzusuchen, gibt es eine maximale Rekursionstiefe, bei der die Suche abgebrochen wird. Wenn diese Rekusionstiefe erreicht wird, muss der Wert des aktuellen Zustands geschätzt werden. Hierfür wird die in diesem Kapitel beschriebene Heuristik verwendet.

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb

Die Klasse `HeuristicWeights` ist eine Hilfsklasse, dessen Instanzen beim Aufruf von der später definierten Funktion `heuristic` übergeben werden können. In dieser Klasse werden alle Gewichtungen für die einzelnen Eigenschaften eines Zustandes gespeichert. Dadurch wird es ermöglicht künstliche Intelligenzen mit verschiedenen Heuristiken gegeneinander antreten zu lassen, um herauszufinden welche Heuristik den Wert eines Zustandes am genausten abbildet.

Eine Instanz der Klasse `HeuristicWeights` besteht aus vier Gewichtungen, eine für jede Eigenschaft:
* `stones`: Diese Eigenschaft zählt, wie viele Steine der Spieler auf dem Spielbrett hat.
* `stash`: Die Steine auf dem Stapel werden ebenfalls gezählt.
* `mills`: Die Anzahl der Mühlen, die ein Spieler auf dem Spielbrett hat, wird durch diese Eigenschaft gezählt.
* `possible_mills`: Die Anzahl der möglichen Mühlen, dh. Mühlen bei denen eine Zelle noch frei ist, werden ebenfalls gezählt. (Siehe `findPossibleMills` im Kapitel *Hilfsfunktionen für die Spielimplementierung* für eine genauere Beschreibung einer möglichen Mühle.)

In [ ]:
class HeuristicWeights():
    def __init__(self, stones=1, stash=1, mills=4, possible_mills=2):
        self.stones = stones
        self.stash = stash
        self.mills = mills
        self.possible_mills = possible_mills

Die Funktion `heuristic` berechnet für einen Spieler den geschätzten Wert eines Zuststandes anhand der oben aufgeführten Eigenschaften. Die Funktion hat drei Argumente:

* `state` $\in States$;
* `player` $\in Players$;
* optional: `weights` ist eine Instanz der Klasse `HeuristicWeights`. 

Für die Implementierung werden alle gewichteten Werte der Eigenschaften für die Spieler weiß `w` und schwarz `b`, sowie der maximale Wert für die Eigenschaft errechnet. Damit die Schätzung des Wertes nicht außerhalb des Wertebereiches, gegeben durch die tatsächlichen Werte für gewinnende (`1.0`) und verlierende (`-1.0`) Zustände, liegt, wird das Maximum um eins erhöht und der errechnete Wert durch das Maximum skaliert. Zum Schluss wird das Vorzeichen angepasst, damit der gegebene Spieler berücksichtigt wird.

In [ ]:
def heuristic(state, player, weights=HeuristicWeights()):
    ((stash_white, stash_black), board) = state
    
    # Count the stones on the board
    white = weights.stones * countStones(state, 'w')
    black = weights.stones * countStones(state, 'b')
    # Count the stones in the stash
    white += weights.stash * stash_white
    black += weights.stash * stash_black
    # There can be at maximum 9 stones per player, so the maximum is the maximum of the weights times the stones
    maximum = 9 * max(weights.stones, weights.stash)
    
    # Count the mills the player currently has
    white += weights.mills * len(findMills(board, 'w'))
    black += weights.mills * len(findMills(board, 'b'))
    # There can be at maximum 4 mills for each player
    maximum += 4 * weights.mills
    
    # Count the possible mills the player currently has
    white += weights.possible_mills * len(findPossibleMills(board, 'w'))
    black += weights.possible_mills * len(findPossibleMills(board, 'b'))
    # There can be at maximum 8 possible mills for each player
    maximum += 8 * weights.possible_mills
    
    # Substract the player scores and clamp them into (-1;+1)
    score = (white - black) / (maximum + 1)

    # Select the correct player
    return score if player == 'w' else -score